# Import group description

In [ ]:
import json as js
with open(r"C:\Users\lunad\Desktop\flickr\group_info\New folder\ginfo_clean_ver.json") as file_input:
        c_ginfo = js.load(file_input)
        group_id = list(c_ginfo.keys())
        c_ginfo = list(c_ginfo.values())

In [ ]:
# Cleaning and processing the texts
import string
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

ascii_chars = set(string.printable)  # speeds things up
def remove_non_ascii_prinatble_from_list(list_of_words):
    return [word for word in list_of_words 
            if all(char in ascii_chars for char in word)]

cc_ginfo = []
for a in range(len(c_ginfo)):
    l = c_ginfo[a]
    l = [i for i in l if 3 <=  len(i) <= 20]
    l = [i for i in l if not any(c.isdigit() for c in i)]
    l = [i.lower() for i in l if i.isalpha()]
    l = remove_non_ascii_prinatble_from_list(l)   
    l = [lemmatizer.lemmatize(i, 'n') for i in l]
    l = [lemmatizer.lemmatize(i, 'v') for i in l]
    cc_ginfo.append(l)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import pandas as pd
import re

# Build TF-IDF Matrix and 
# Calculate Euclidean distance for each pair of group

In [ ]:
vect = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
tfidf_matrix = vect.fit_transform(cc_ginfo)
e_distance = euclidean_distances(tfidf_matrix)

In [ ]:
# Store in dataframe
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index = group_id, columns = vect.get_feature_names())
ed_df = pd.DataFrame(e_distance, index = group_id, columns = group_id)#, index = group_id, columns = vect.get_feature_names())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

In [ ]:
# Visualize the euclidean distance
plt.imshow(e_distance, zorder=2, cmap='Blues', interpolation='nearest')
plt.colorbar()

# PCA
## 10 Components

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%config InlineBackend.figure_format='retina'

# Standardize the data to have a mean of ~0 and a variance of 1
#X_std = StandardScaler().fit_transform(df)
# Create a PCA instance: pca
pca = PCA(n_components=10)
w_principalComponents = pca.fit_transform(tfidf_df)
w_features = range(pca.n_components)

## Visualize the variance

In [ ]:
plt.bar(w_features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(w_features)

In [ ]:
# These 10 components only explained 10% variance
pca.explained_variance_ratio_.cumsum()

In [ ]:
# The first two components
PCA_components = pd.DataFrame(w_principalComponents)
plt.scatter(PCA_components[0], PCA_components[1], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

# PCA on Bag of Words
## Building DTM

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

mlb = MultiLabelBinarizer()
data = cc_ginfo
array = mlb.fit_transform(data)
classes = mlb.classes_

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%config InlineBackend.figure_format='retina'

bow_df = pd.DataFrame(array, index = group_id, columns = classes)

# Standardize the data to have a mean of ~0 and a variance of 1
whole_X_std = StandardScaler().fit_transform(bow_df)
# Create a PCA instance: pca
whole_pca = PCA(n_components=10)
w_principalComponents = whole_pca.fit_transform(whole_X_std)
w_features = range(pca.n_components)

In [ ]:
plt.bar(w_features, whole_pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(w_features)

In [ ]:
# These 10 components only explained 10% variance
whole_pca.explained_variance_ratio_.cumsum()

## DTM with terms occur more then 1 time

In [ ]:
bow_df.loc['Total',:]= bow_df.sum(axis=0)

In [ ]:
bow_df = bow_df.transpose()
nbow_df = bow_df[bow_df['Total']>1]

In [ ]:
nbow_df = nbow_df.transpose()

In [ ]:
nbow_df = nbow_df.drop(["Total"])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%config InlineBackend.figure_format='retina'

# Standardize the data to have a mean of ~0 and a variance of 1
morethan1_X_std = StandardScaler().fit_transform(nbow_df)
# Create a PCA instance: pca
morethan1_pca = PCA(n_components=10)
morethan1_principalComponents = morethan1_pca.fit_transform(morethan1_X_std)
morethan1_features = range(morethan1_pca.n_components)

In [ ]:
plt.bar(morethan1_features, morethan1_pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(morethan1_features)

In [ ]:
# These 10 components only explained 13% variance
morethan1_pca.explained_variance_ratio_.cumsum()

# Filter the TF-IDF Matrix based on its score

In [ ]:
def Average(lst): 
    return sum(lst) / len(lst) 

In [ ]:
# Calculate the total score for each word
tfidf_df.loc['Total',:]= tfidf_df.sum(axis=0)
t_df = tfidf_df.transpose()
# Calculate the mean of the total scores of the words
avg = Average(t_df['Total'].tolist())
abavg_tfidf_df = t_df[t_df['Total']>=avg]

In [ ]:
# Generate a new tf-idf matrix with total score more than the mean
abavg_tfidf_df = abavg_tfidf_df.transpose()
abavg_tfidf_df = abavg_tfidf_df.drop(["Total"])


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%config InlineBackend.figure_format='retina'

# Standardize the data to have a mean of ~0 and a variance of 1
# above_avg_X_std = StandardScaler().fit_transform(x_df)
# Create a PCA instance: pca
above_avg_pca = PCA(n_components=10)
above_avg_w_principalComponents = above_avg_pca.fit_transform(abavg_tfidf_df)
above_avg_w_features = range(above_avg_pca.n_components)

In [ ]:
plt.bar(above_avg_w_features, above_avg_pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(above_avg_w_features)

In [ ]:
# These 10 components only explained 13% variance
above_avg_pca.explained_variance_ratio_.cumsum()

In [ ]:
s_PCA_components = pd.DataFrame(above_avg_w_principalComponents)
plt.scatter(s_PCA_components[0], s_PCA_components[1], alpha=.1, color='black')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

# MDS

In [ ]:
from sklearn.manifold import MDS
model = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
out = model.fit_transform(e_distance)
#results = model.fit(ed_df.values)

In [ ]:
colorize = dict(c=e_distance[:, 0], cmap=plt.cm.get_cmap('rainbow', 5))
plt.scatter(out[:, 0], out[:, 1], **colorize)
plt.axis('equal')